In [1]:
# Combine all the data, both games and game details
import aiohttp
import asyncio
import pandas as pd

async def fetch_game_details(session, appid):
    async with session.get(f"https://steamspy.com/api.php?request=appdetails&appid={appid}") as resp:
        return await resp.json()

async def fetch_games(session):
    async with session.get("https://api.steampowered.com/ISteamApps/GetAppList/v2") as resp:
        return await resp.json()

async def save_games_to_csv(session,games):
    games_data = []
    for game in games["applist"]["apps"]:
        game_details = await fetch_game_details(session, game["appid"])
        if game_details:
            data = game_details
            games_data.append((game["appid"], game["name"], data.get("price", "0"), data.get("developer", ""), data.get("publisher", ""), data.get("average_forever", 0), data.get("median_forever", 0), data.get("ccu", 0), data.get("genre", [])))

    df = pd.DataFrame(games_data, columns=["appid", "name", "price", "developer", "publisher", "average_forever", "median_forever", "ccu", "genres"])
    df.to_csv("all_games.csv", index=False)
    return df

async def main():
    async with aiohttp.ClientSession() as session:
        games = await fetch_games(session)
        await save_games_to_csv(session, games)

if __name__ == "__main__":
    await main()


CancelledError: 